In [1]:
import os
# Specify the new directory path
parent_directory = 'D:/Projects/CornYield/src'

# Change the current directory
os.chdir(parent_directory) 
# %pwd

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from dataset import get_filename_metadata
from dl.dl_dataset import MixedDataset
from dl.train import train_with_cross_validation, train, validate, data_transform

In [5]:
from dl.model import ResNetRegression

In [6]:
yield_file = 'D:/Corn_Yield/BL2022_Yld.csv'
weather_file = 'D:/Corn_Yield/CoAgMet ET_VBA_calc_2022.xlsm'

# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220616_DOY167_extracted_filled'
# out_path = '../output/DOY167/'
# doy = 167
# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220705_DOY186_extracted_filled'
# out_path = '../output/DOY186/'
# doy = 186
# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220720_DOY201_extracted_filled'
# out_path = '../output/DOY201/'


In [7]:
img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220628_DOY179_extracted_filled'
out_path = '../output/DOY179/'
doy = 179

In [8]:
weather_pf = pd.read_excel(weather_file,sheet_name=1, header=[0,1,3])

header_name = weather_pf.columns
selected_rows = weather_pf[(weather_pf[header_name[-1]] > doy-6) & (weather_pf[header_name[-1]] < doy+6)]

air_temperature = selected_rows[header_name[2]].mean()
precipitation = selected_rows[header_name[9]].mean()
soil_temperature = selected_rows[header_name[10]].mean()
soil_temperature2 = selected_rows[header_name[11]].mean()

In [9]:
key_word_list = ['Ref_filled.tif']
for keyword in key_word_list:

    img_list, yield_pf = get_filename_metadata(img_path, yield_file, keyword)
    
    yield_list = list(yield_pf['Yield_Bu_Ac'])

    # metadata = yield_pf[['Variety_int', 'Irrigation_int']]
    metadata = yield_pf[['Variety_int', 'Irrigation_int']]/3
    metadata['DOY'] = doy / 366
    metadata['Month'] = (doy / 30)/12
    metadata['Stage'] = (doy/30 - 5)/5
    metadata['air_temperature'] = air_temperature
    metadata['precipitation'] = precipitation
    metadata['soil_temperature'] = soil_temperature
    metadata['soil_temperature2'] = soil_temperature2

    indices = yield_pf.index[yield_pf['Variety_int'] == 2].tolist()
    yield_list = [yield_list[i] for i in indices]
    img_list = [img_list[i] for i in indices]
    metadata = metadata[metadata['Variety_int'] == 2/3]
    metadata = metadata.reset_index(drop=True)

# new_df = df[selected_columns]
    # for suffix_list in suffix_list_list:
    total_size = len(img_list)
    train_size = int(0.8 * total_size)  # 80% for training
    test_size = int(0.2 * total_size)   # 20% for validation

    # VI_list = ['evi', 'ndvi']
    # suffix_list = ['LWIR_filled.tif']
# 
    VI_list = None
    suffix_list = None
    # print(metadata)
    # train_val_dataset = MixedDataset(img_list, yield_list, metadata)

        # Use train_test_split to split the indices into training and testing sets
    train_indices, test_indices = train_test_split(range(len(img_list)), test_size=test_size, random_state=39)

    train_val_dataset = MixedDataset([img_list[i] for i in train_indices], [yield_list[i] for i in train_indices], metadata.loc[train_indices], VI_list=VI_list, suffix_list=suffix_list,  transform=data_transform())
    test_dataset = MixedDataset([img_list[i] for i in test_indices], [yield_list[i] for i in test_indices], metadata.loc[test_indices], suffix_list=suffix_list, VI_list=VI_list)



2. Extract Features

In [10]:
import torch
from dl.model import ResNetFNN

In [11]:
# state_dict = "path/model_ResNetRegression_DOY167_Batch=32_state.pth"
# in_channel = 5
# batch_size = 32

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)

# loaded_model = ResNetRegression(in_channel, 1)
# loaded_model.load_state_dict(torch.load(state_dict))

# # modules = list(loaded_model.resnet.children())[:-1]
# # loaded_model=torch.nn.Sequential(*modules)
# # Define a new function for the specific instance
# def custom_forward(self, x):
#     x = self.resnet.conv1(x)
#     x = self.resnet.bn1(x)
#     x = self.resnet.relu(x)
#     x = self.resnet.maxpool(x)

#     x = self.resnet.layer1(x)
#     x = self.resnet.layer2(x)
#     x = self.resnet.layer3(x)
#     x = self.resnet.layer4(x)

#     x = self.resnet.avgpool(x)
#     x = x.view(x.size(0), -1)
#     # x = self.resnet.fc(x)
#     return x
# loaded_model.forward = custom_forward
# loaded_model.to(device)
# # Assign the custom function to the instance
# # obj.custom_function = custom_function
# loaded_model.eval()


In [12]:
import torch.optim as optim

in_channel = 5
num_epochs = 120
batch_size = 32

# Initialize an empty list to store fold-wise performance
fold_accuracies = []

# Initialize a new model for each fold
# model = CNNRegression(in_channel)
model = ResNetFNN(in_channel,9,1)
# model = EncoderCNN(in_channel, 1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)


criterion = torch.nn.MSELoss()  # Mean Squared Error loss function
# optimizer = optim.Adam(list(conv.parameters()) + list(deconv.parameters()), lr=0.001)  # Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer



cuda


In [13]:

model = train_with_cross_validation(model, train_val_dataset, batch_size, num_epochs, optimizer, criterion, is_dual_data=True)

model_name = "path/model_metadata_" + model.__class__.__name__+ img_path[-24:-17] +"_Batch=" +str(batch_size) + "_state.pth"
torch.save(model.state_dict(), model_name)

validation mse is 684.4481643676758
validation mse is 507.3188758850098
validation mse is 293.1145896911621
validation mse is 361.95020904541013
validation mse is 428.4539619445801


In [14]:
test_accuracy, test_prediction = validate(model, test_dataset,criterion, batch_size = batch_size, is_return_output = True, is_dual_data=True)
test_accuracy = np.sqrt(np.mean(test_accuracy))
print(f'validation mse is {test_accuracy}')

validation mse is 19.249476279528565


In [15]:
from ml.ml_predict import plot_result_separate
yield_data = np.array(yield_list)
irrigate_data = np.array(yield_pf['Irrigation_int'])

test_truth = yield_data[test_indices]


name_tag = img_path[-23:-17]
out_name = name_tag + ' ' + 'Pioneer' + ' ' 
out_name = out_name + keyword[:-11].upper() 
out_name = out_name + '+ Meta CNN'

plot_result_separate(np.array(test_truth), np.array(test_prediction), test_indices, irrigate_data, out_name)

DOY179 Pioneer REF+ Meta CNN  r-squared = 0.8401507853290271, rmse = 19.25511577411272


In [ ]:
# def plot_result_metadata():

In [ ]:
# # for i in test_indices:
# #     img = 
# # batch_size = 1
# train_loader = torch.utils.data.DataLoader(train_val_dataset, batch_size=batch_size, shuffle=True)
# img_features = []
# yield_labels = []
# for i, data in enumerate(train_loader, 0):
#     inputs, labels = data
#     labels = labels.view(-1, 1)\








#     # print(img.shape)
#     # print(metadata.shape)
#     # print(labels.shape)
#     img = img.to(device)
#     metadata = metadata.to(device)
#     labels = labels.to(device)
    
#     # Zero the parameter gradients
#     optimizer.zero_grad()
#     # Forward pass
#     outputs = model(img, metadata)
    
#     # Compute loss
#     if  outputs.shape == labels.shape:
#         loss = criterion(outputs, labels)
        
#         # Backward pass and optimization
#         loss.backward()
#         # update weights
#         optimizer.step()
#     else:
#         print('outputs shape is ', outputs.shape, ' the size is ', outputs.size())
#         print('labels shape is ', labels.shape, ' the size is ', labels.size())
                

    

#     # Forward pass
#     img_feature = loaded_model(loaded_model, inputs)
#     img_features.append(img_feature.squeeze().tolist())
#     yield_labels.append(labels.squeeze().tolist())
    
# print(len(img_features))
# img_features = np.array(img_features)
# yield_labels = np.array(yield_labels)
# print(img_features.shape, yield_labels.shape)
# original_indices = np.arange(len(yield_labels))
# train_images, test_images, train_yields, test_yields, train_indices, test_indices = train_test_split(
#     img_features, yield_list, original_indices, test_size=0.2, random_state=39)

# all_features = []

# img_features = loaded_model()
# for img_file in img_list:
#     img = read_img(img_file, VI_list = VI_list, 
#                     suffix_list = suffix_list, is_vi_only=is_vi_only)    
#     all_image.append(img)

# return np.array(all_image)